## Model Preparation

In [61]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [62]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.ensemble import ExtraTreesClassifier

In [63]:
full_data_clean = pd.read_csv('full_data_clean.csv')
full_data_clean.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeID,Gender,...,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement,PerformanceRating
0,51,No,Travel_Rarely,Sales,6,2,Life Sciences,1,1,Female,...,1.0,6,1,0,0,3.0,4.0,2.0,3,3
1,31,Yes,Travel_Frequently,Research & Development,10,1,Life Sciences,1,2,Female,...,6.0,3,5,1,4,3.0,2.0,4.0,2,4
2,32,No,Travel_Frequently,Research & Development,17,4,Other,1,3,Male,...,5.0,2,5,0,3,2.0,2.0,1.0,3,3
3,38,No,Non-Travel,Research & Development,2,5,Life Sciences,1,4,Male,...,13.0,5,8,7,5,4.0,4.0,3.0,2,3
4,32,No,Travel_Rarely,Research & Development,10,1,Medical,1,5,Male,...,9.0,2,6,0,4,4.0,1.0,3.0,3,3


In [64]:
full_data_clean.shape

(4410, 29)

In [65]:
full_data_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4410 entries, 0 to 4409
Data columns (total 29 columns):
Age                        4410 non-null int64
Attrition                  4410 non-null object
BusinessTravel             4410 non-null object
Department                 4410 non-null object
DistanceFromHome           4410 non-null int64
Education                  4410 non-null int64
EducationField             4410 non-null object
EmployeeCount              4410 non-null int64
EmployeeID                 4410 non-null int64
Gender                     4410 non-null object
JobLevel                   4410 non-null int64
JobRole                    4410 non-null object
MaritalStatus              4410 non-null object
MonthlyIncome              4410 non-null int64
NumCompaniesWorked         4410 non-null float64
Over18                     4410 non-null object
PercentSalaryHike          4410 non-null int64
StandardHours              4410 non-null int64
StockOptionLevel           4410 non-n

In [66]:
%%time
def summary_table(df):
    summary = pd.DataFrame(dict(dataFeatures = df.columns,
                                  dataType = df.dtypes,
                                  null = df.isna().sum(),
                                  null_percentage = round(df.isna().sum() / len(df) *100,2),
                                  unique = df.nunique(),
#                                   uniqueSample = [list(df[i].drop_duplicates().sample(2)) for i in df.columns]
                               )
                           ).reset_index(drop=True)
    summary['unique_percentage'] = round(summary['unique'] / len(df) *100,2) #convert unique to percentage values
    return summary

summary_table(full_data_clean)

Wall time: 72.8 ms


,dataFeatures,dataType,null,null_percentage,unique,unique_percentage
0,Age,int64,0,0.0,43,0.98
1,Attrition,object,0,0.0,2,0.05
2,BusinessTravel,object,0,0.0,3,0.07
3,Department,object,0,0.0,3,0.07
4,DistanceFromHome,int64,0,0.0,29,0.66
5,Education,int64,0,0.0,5,0.11
6,EducationField,object,0,0.0,6,0.14
7,EmployeeCount,int64,0,0.0,1,0.02
8,EmployeeID,int64,0,0.0,4410,100.00
9,Gender,object,0,0.0,2,0.05


In [67]:
full_data_clean[['Education',
           'EmployeeCount',
           'EmployeeID',
           'JobLevel',
           'StockOptionLevel','EnvironmentSatisfaction',
           'JobSatisfaction',
           'WorkLifeBalance',
           'JobInvolvement',
           'PerformanceRating']] = full_data_clean[['Education',
                                              'EmployeeCount',
                                              'EmployeeID',
                                              'JobLevel',
                                              'StockOptionLevel','EnvironmentSatisfaction',
                                              'JobSatisfaction',
                                              'WorkLifeBalance',
                                              'JobInvolvement',
                                              'PerformanceRating']].astype('object')

In [68]:
summary_table(full_data_clean) #check for datatypes again after conversion

,dataFeatures,dataType,null,null_percentage,unique,unique_percentage
0,Age,int64,0,0.0,43,0.98
1,Attrition,object,0,0.0,2,0.05
2,BusinessTravel,object,0,0.0,3,0.07
3,Department,object,0,0.0,3,0.07
4,DistanceFromHome,int64,0,0.0,29,0.66
5,Education,object,0,0.0,5,0.11
6,EducationField,object,0,0.0,6,0.14
7,EmployeeCount,object,0,0.0,1,0.02
8,EmployeeID,object,0,0.0,4410,100.00
9,Gender,object,0,0.0,2,0.05


In [36]:
tar_feat = ['Attrition']
num_features = ['Age','DistanceFromHome','MonthlyIncome','NumCompaniesWorked','PercentSalaryHike',
                'StandardHours','TotalWorkingYears','TrainingTimesLastYear','YearsAtCompany','YearsSinceLastPromotion',
                'YearsWithCurrManager']
cat_features = ['BusinessTravel','Department','Education','EducationField',
                'EmployeeCount','EmployeeID','EnvironmentSatisfaction','Gender','JobInvolvement',
                'JobLevel','JobRole','JobSatisfaction','MaritalStatus','Over18','PerformanceRating','StockOptionLevel',
               'WorkLifeBalance']

##option manual

In [69]:
categorical_features = list(full_data_clean.select_dtypes(include = [object]))
print(len(categorical_features))

numerical_features = list(full_data_clean.select_dtypes(exclude = [object]))
print(len(numerical_features))

target_features = categorical_features.remove('Attrition')
#option otomatis generate tapi perlu list satu per satu features
# total is 29 Features/Columns
categorical_features

18
11


['BusinessTravel',
 'Department',
 'Education',
 'EducationField',
 'EmployeeCount',
 'EmployeeID',
 'Gender',
 'JobLevel',
 'JobRole',
 'MaritalStatus',
 'Over18',
 'StockOptionLevel',
 'EnvironmentSatisfaction',
 'JobSatisfaction',
 'WorkLifeBalance',
 'JobInvolvement',
 'PerformanceRating']

Some categorical features are only in number format, which implies that the rest of those that are still non-number format would have to be converted/encoded to number format

In [70]:
full_data_clean['Department'].unique()

array(['Sales', 'Research & Development', 'Human Resources'], dtype=object)

In [71]:
from sklearn.preprocessing import LabelEncoder

cat_change_feature = ['Attrition','BusinessTravel','Department','EducationField',
                      'Gender','JobRole','MaritalStatus','Over18']
LE = LabelEncoder()

for i in cat_change_feature:
    full_data_clean[i] = LE.fit_transform(full_data_clean[i])


In [72]:
full_data_clean.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeID,Gender,...,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement,PerformanceRating
0,51,0,2,2,6,2,1,1,1,0,...,1.0,6,1,0,0,3,4,2,3,3
1,31,1,1,1,10,1,1,1,2,0,...,6.0,3,5,1,4,3,2,4,2,4
2,32,0,1,1,17,4,4,1,3,1,...,5.0,2,5,0,3,2,2,1,3,3
3,38,0,0,1,2,5,1,1,4,1,...,13.0,5,8,7,5,4,4,3,2,3
4,32,0,2,1,10,1,3,1,5,1,...,9.0,2,6,0,4,4,1,3,3,3


In [73]:
full_data_clean.to_csv('full_data_clean_enc.csv', index = False) #set file to full_data_clean_enc

In [74]:
full_data_clean_enc = pd.read_csv('full_data_clean_enc.csv')
full_data_clean_enc.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeID,Gender,...,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement,PerformanceRating
0,51,0,2,2,6,2,1,1,1,0,...,1.0,6,1,0,0,3.0,4.0,2.0,3,3
1,31,1,1,1,10,1,1,1,2,0,...,6.0,3,5,1,4,3.0,2.0,4.0,2,4
2,32,0,1,1,17,4,4,1,3,1,...,5.0,2,5,0,3,2.0,2.0,1.0,3,3
3,38,0,0,1,2,5,1,1,4,1,...,13.0,5,8,7,5,4.0,4.0,3.0,2,3
4,32,0,2,1,10,1,3,1,5,1,...,9.0,2,6,0,4,4.0,1.0,3.0,3,3


In [75]:
full_data_clean_enc.shape #in terms of number of rows and columns, they are still the same as before, but data has been transformed

(4410, 29)

In [76]:
full_data_clean_enc.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Age,4410.0,36.923810,9.133301,18.0,30.00,36.0,43.00,60.0
Attrition,4410.0,0.161224,0.367780,0.0,0.00,0.0,0.00,1.0
BusinessTravel,4410.0,1.607483,0.665304,0.0,1.00,2.0,2.00,2.0
Department,4410.0,1.260544,0.527673,0.0,1.00,1.0,2.00,2.0
DistanceFromHome,4410.0,9.192517,8.105026,1.0,2.00,7.0,14.00,29.0
Education,4410.0,2.912925,1.023933,1.0,2.00,3.0,4.00,5.0
EducationField,4410.0,2.247619,1.331067,0.0,1.00,2.0,3.00,5.0
EmployeeCount,4410.0,1.000000,0.000000,1.0,1.00,1.0,1.00,1.0
EmployeeID,4410.0,2205.500000,1273.201673,1.0,1103.25,2205.5,3307.75,4410.0
Gender,4410.0,0.600000,0.489954,0.0,0.00,1.0,1.00,1.0


In [77]:
ind_features = numerical_features + categorical_features #features other than target (independent)

In [78]:
from sklearn.preprocessing import RobustScaler

RS = RobustScaler()
full_data_clean_enc[ind_features] = RS.fit_transform(full_data_clean_enc[ind_features])

In [79]:
full_data_clean_enc.to_csv('full_data_clean_enc_rb.csv', index = False)
#set another dataset for later use if one were to use the robust scaled data